In [1]:
!pip install transformers
!pip install torch

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import torch
import transformers as transf
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('Restaurant reviews.csv')

In [4]:
df.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN


In [5]:
df.shape

(10000, 8)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Restaurant  10000 non-null  object
 1   Review      9955 non-null   object
 2   Rating      9962 non-null   object
 3   Time        9962 non-null   object
dtypes: object(4)
memory usage: 312.6+ KB


In [6]:
df = df.drop(["Reviewer", "Metadata", "Pictures" ,"7514"], axis=1)
df.head()

,Restaurant,Review,Rating,Time
0,Beyond Flavours,"The ambience was good, food was quite good . h...",5,5/25/2019 15:54
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,5,5/25/2019 14:20
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,5,5/24/2019 22:54
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,5,5/24/2019 22:11
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,5,5/24/2019 21:37


In [7]:
#df[3].value_counts()

In [8]:
#df_sample = df.sample(2000)
#df_sample[1].value_counts()

1 - Charger le modèle Bart

In [7]:
model_class, tokenizer_c, weights_pretrained = (transf.DistilBertModel, transf.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_c.from_pretrained(weights_pretrained)
model = model_class.from_pretrained(weights_pretrained)



2 - Préparer le texte

In [8]:
exemple = df.head(5)["Review"].apply(str).tolist()

count = 1
for review in exemple:
    print(f"Review numéro {count} : {review}")
    count+=1

Review numéro 1 : The ambience was good, food was quite good . had Saturday lunch , which was cost effective .
Good place for a sate brunch. One can also chill with friends and or parents.
Waiter Soumen Das was really courteous and helpful.
Review numéro 2 : Ambience is too good for a pleasant evening. Service is very prompt. Food is good. Over all a good experience. Soumen Das - kudos to the service
Review numéro 3 : A must try.. great food great ambience. Thnx for the service by Pradeep and Subroto. My personal recommendation is Penne Alfredo Pasta:) ....... Also the music in the background is amazing.
Review numéro 4 : Soumen das and Arun was a great guy. Only because of their behavior and sincerety, And good food off course, I would like to visit this place again.
Review numéro 5 : Food is good.we ordered Kodi drumsticks and basket mutton biryani. All are good. Thanks to Pradeep. He served well. We enjoyed here. Ambience is also very good.


In [9]:
exemple = df.head(300)["Review"].apply(str).tolist()
print(exemple)
print(len(exemple))



['The ambience was good, food was quite good . had Saturday lunch , which was cost effective .\nGood place for a sate brunch. One can also chill with friends and or parents.\nWaiter Soumen Das was really courteous and helpful.', 'Ambience is too good for a pleasant evening. Service is very prompt. Food is good. Over all a good experience. Soumen Das - kudos to the service', 'A must try.. great food great ambience. Thnx for the service by Pradeep and Subroto. My personal recommendation is Penne Alfredo Pasta:) ....... Also the music in the background is amazing.', 'Soumen das and Arun was a great guy. Only because of their behavior and sincerety, And good food off course, I would like to visit this place again.', 'Food is good.we ordered Kodi drumsticks and basket mutton biryani. All are good. Thanks to Pradeep. He served well. We enjoyed here. Ambience is also very good.', 'Ambiance is good, service is good, food is aPradeecp and subro best service\nFood is good. Papiya good hostess an

In [10]:
count = 1
for review in exemple:
    print(f"Review numéro {count} : ", tokenizer.encode(review, add_special_tokens = True))
    count+=1


Review numéro 1 :  [101, 1996, 2572, 11283, 5897, 2001, 2204, 1010, 2833, 2001, 3243, 2204, 1012, 2018, 5095, 6265, 1010, 2029, 2001, 3465, 4621, 1012, 2204, 2173, 2005, 1037, 2938, 2063, 7987, 4609, 2818, 1012, 2028, 2064, 2036, 10720, 2007, 2814, 1998, 2030, 3008, 1012, 15610, 2061, 27417, 8695, 2001, 2428, 2457, 14769, 1998, 14044, 1012, 102]
Review numéro 2 :  [101, 2572, 11283, 5897, 2003, 2205, 2204, 2005, 1037, 8242, 3944, 1012, 2326, 2003, 2200, 25732, 1012, 2833, 2003, 2204, 1012, 2058, 2035, 1037, 2204, 3325, 1012, 2061, 27417, 8695, 1011, 13970, 12269, 2000, 1996, 2326, 102]
Review numéro 3 :  [101, 1037, 2442, 3046, 1012, 1012, 2307, 2833, 2307, 2572, 11283, 5897, 1012, 16215, 26807, 2005, 1996, 2326, 2011, 10975, 9648, 13699, 1998, 4942, 21709, 2080, 1012, 2026, 3167, 12832, 2003, 9502, 2063, 19423, 24857, 1024, 1007, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 2036, 1996, 2189, 1999, 1996, 4281, 2003, 6429, 1012, 102]
Review numéro 4 :  [101, 2061, 27417, 8695, 1998, 28217,

In [11]:
count = 1
for review in exemple:
    print(f"Review numéro {count} : ", tokenizer.tokenize(review))
    count+=1

Review numéro 1 :  ['the', 'am', '##bie', '##nce', 'was', 'good', ',', 'food', 'was', 'quite', 'good', '.', 'had', 'saturday', 'lunch', ',', 'which', 'was', 'cost', 'effective', '.', 'good', 'place', 'for', 'a', 'sat', '##e', 'br', '##un', '##ch', '.', 'one', 'can', 'also', 'chill', 'with', 'friends', 'and', 'or', 'parents', '.', 'waiter', 'so', '##umen', 'das', 'was', 'really', 'court', '##eous', 'and', 'helpful', '.']
Review numéro 2 :  ['am', '##bie', '##nce', 'is', 'too', 'good', 'for', 'a', 'pleasant', 'evening', '.', 'service', 'is', 'very', 'prompt', '.', 'food', 'is', 'good', '.', 'over', 'all', 'a', 'good', 'experience', '.', 'so', '##umen', 'das', '-', 'ku', '##dos', 'to', 'the', 'service']
Review numéro 3 :  ['a', 'must', 'try', '.', '.', 'great', 'food', 'great', 'am', '##bie', '##nce', '.', 'th', '##nx', 'for', 'the', 'service', 'by', 'pr', '##ade', '##ep', 'and', 'sub', '##rot', '##o', '.', 'my', 'personal', 'recommendation', 'is', 'penn', '##e', 'alfredo', 'pasta', ':', 

In [13]:
tokenized = df.head(300)["Review"].apply(str).apply((lambda x : tokenizer.encode(x,add_special_tokens = True, max_length=512, truncation= True)))
print(tokenized)

0      [101, 1996, 2572, 11283, 5897, 2001, 2204, 101...
1      [101, 2572, 11283, 5897, 2003, 2205, 2204, 200...
2      [101, 1037, 2442, 3046, 1012, 1012, 2307, 2833...
3      [101, 2061, 27417, 8695, 1998, 28217, 2001, 10...
4      [101, 2833, 2003, 2204, 1012, 2057, 3641, 1284...
                             ...                        
295    [101, 4669, 1996, 2833, 2009, 1005, 1055, 2001...
296    [101, 2028, 1997, 1996, 2190, 7884, 1999, 2237...
297    [101, 29400, 2024, 2025, 2061, 2919, 6090, 325...
298    [101, 2065, 1057, 1054, 2559, 2005, 1037, 9379...
299    [101, 2307, 2572, 11283, 5897, 1998, 8403, 283...
Name: Review, Length: 300, dtype: object


In [14]:
max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)

print(max_len)

449


In [15]:
tokenized_zeroes = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
np.array(tokenized_zeroes).shape

(300, 449)

In [16]:
attention_mask = np.where(tokenized_zeroes != 0, 1, 0)
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [17]:
attention_mask.shape

(300, 449)

In [18]:
input_ids = torch.tensor(np.array(tokenized_zeroes))
attention_mask = torch.tensor(attention_mask)

input_ids

tensor([[  101,  1996,  2572,  ...,     0,     0,     0],
        [  101,  2572, 11283,  ...,     0,     0,     0],
        [  101,  1037,  2442,  ...,     0,     0,     0],
        ...,
        [  101, 29400,  2024,  ...,     0,     0,     0],
        [  101,  2065,  1057,  ...,     0,     0,     0],
        [  101,  2307,  2572,  ...,     0,     0,     0]])

In [19]:
with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask = attention_mask)

In [20]:
features = last_hidden_states[0][:,0,:]
features.shape

torch.Size([300, 768])

In [34]:
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df['rating_shaped'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)
df.head()


,Restaurant,Review,Rating,Time,rating_shaped
0,Beyond Flavours,"The ambience was good, food was quite good . h...",5.0,5/25/2019 15:54,1
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,5.0,5/25/2019 14:20,1
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,5.0,5/24/2019 22:54,1
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,5.0,5/24/2019 22:11,1
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,5.0,5/24/2019 21:37,1


In [35]:
labels = df.head(300)["rating_shaped"]

In [36]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

lr_clf.score(test_features, test_labels)

0.9733333333333334

# Conclusion :

# Précision = 96% au deuxième essai
# Précision = 97,3% au deuxième essai
# On est beaucoup plus précis que sur notre LSTM classique réalisé au TP1

In [37]:
text = tokenizer.encode("it's a good product, but i don't like it", add_special_tokens = True)
tokenized.zeroes = np.array([text + [0]*(max_len - len(text))])
inputs_ids = torch.tensor(tokenized_zeroes)
attention_mask = torch.tensor(np.where(tokenized_zeroes != 0, 1, 0))
with torch.no_grad():
  pred_hidden_states = model(inputs_ids, attention_mask = attention_mask)
features = pred_hidden_states[0][:,0,:]

lr_clf.predict(features)
lr_clf.predict_proba(features)


array([[1.08419063e-02, 9.89158094e-01],
       [7.28268748e-03, 9.92717313e-01],
       [9.53607383e-03, 9.90463926e-01],
       [1.74838158e-02, 9.82516184e-01],
       [3.65586511e-03, 9.96344135e-01],
       [2.80470914e-03, 9.97195291e-01],
       [2.03744519e-02, 9.79625548e-01],
       [3.15866943e-02, 9.68413306e-01],
       [4.67227719e-04, 9.99532772e-01],
       [2.47679757e-03, 9.97523202e-01],
       [3.01198393e-03, 9.96988016e-01],
       [9.32389670e-04, 9.99067610e-01],
       [5.72153078e-03, 9.94278469e-01],
       [1.86233050e-02, 9.81376695e-01],
       [8.65179888e-01, 1.34820112e-01],
       [1.48612853e-02, 9.85138715e-01],
       [1.96605982e-02, 9.80339402e-01],
       [1.66503246e-02, 9.83349675e-01],
       [2.47993214e-03, 9.97520068e-01],
       [1.48373573e-02, 9.85162643e-01],
       [4.33366290e-03, 9.95666337e-01],
       [1.71232040e-01, 8.28767960e-01],
       [1.43732968e-02, 9.85626703e-01],
       [1.81406046e-03, 9.98185940e-01],
       [7.583022